In [2]:
# Mount Google Driver
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive
drive.mount(ROOT)           # we mount the google drive at /content/drive
# change to clrs directionary
%cd "/content/drive/My Drive/Colab Notebooks/fluent_python_notes"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/fluent_python_notes


In [ ]:
%mkdir ch19
%mkdir ch19/data
!touch ch19/__init__.py

mkdir: cannot create directory ‘ch19’: File exists


## 19.0 序言

- Python 中,数据的属性和处理数据的方法统称属性(attribute)
  - 方法只是可调用的属性
- 除了这二者之外, 还可以创建特性(property), 在不改变类接口的前提下, 使用存取方法(即读值方法和设值方法)修改数据属性
  - 这与统一访问原则相符
  > 不管服务是由存储还是计算实现的, 一个模块提供的所有服务都应该通过统一的方式使用

## 19.1 使用动态属性转换数据

###### 示例 19-1 osconfeed.json 文件中的记录示例;节略了部分字段的内容

```json
{ "Schedule":
  { "conferences": [{"serial": 115 }],
    "events": [
      { "serial": 34505,
        "name": "Why Schools Don ́t Use Open Source to Teach Programming",
        "event_type": "40-minute conference session",
        "time_start": "2014-07-23 11:30:00",
        "time_stop": "2014-07-23 12:10:00",
        "venue_serial": 1462,
        "description": "Aside from the fact that high school programming...",
        "website_url": "http://oscon.com/oscon2014/public/schedule/detail/34505",
        "speakers": [157509],
        "categories": ["Education"] }
    ],
    "speakers": [
      { "serial": 157509,
        "name": "Robert Lefkowitz",
        "photo": null,
        "url": "http://sharewave.com/",
        "position": "CTO",
        "affiliation": "Sharewave",
        "twitter": "sharewaveteam",
        "bio": "Robert  ́r0ml ́ Lefkowitz is the CTO at Sharewave, a startup..." }
    ],
    "venues": [
      { "serial": 1462,
        "name": "F151",
        "category": "Conference Venues" }
    ]
  }
}
```

- 整个数据集是一个 JSON 对象,里面有一个键,名为 `Schedule`
  - 这个键对应的值也是一个映像,有 4 个键: `conferences`、`events`、`speakers` 和 `venues`
    - 这 4 个键对应的值都是一个记录列表
    - 这 4 个列表中的每个元素都有一个名为 `serial` 的字段,这是元素在各个列表中的唯一标识符

###### 示例 19-2 osconfeed.py:下载 osconfeed.json

In [ ]:
%%writefile ch19/osconfeed.py
from urllib.request import urlopen
import warnings
import os
import json

URL = 'http://www.oreilly.com/pub/sc/osconfeed'
JSON = "ch19/data/osconfeed.json"


def load():
  if not os.path.exists(JSON):
    msg = 'download {} to {}'.format(URL, JSON)
    warnings.warn(msg)  # 如需下载，就发出提醒
    with urlopen(URL) as remote, open(JSON, 'wb') as local:  # 在 with 语句中使用两个上下文管理器，分别用于读取和保存远程文件
      local.write(remote.read())
  
  with open(JSON) as fp:
    return json.load(fp)  # 解析 JSON 文件，返回 Python 原生对象


Overwriting ch19/osconfeed.py


###### 示例 19-3 osconfeed.py: 示例 19-2 的 doctest

In [ ]:
from ch19.osconfeed import load

In [ ]:
feed = load()

/content/drive/My Drive/Colab Notebooks/fluent_python_notes/ch19/osconfeed.py:13: UserWarning: download http://www.oreilly.com/pub/sc/osconfeed to ch19/data/osconfeed.json
  warnings.warn(msg)  # 如需下载，就发出提醒


In [ ]:
# 显示 "Schedule" 中的 4 个记录集合
sorted(feed['Schedule'].keys())

['conferences', 'events', 'speakers', 'venues']

In [ ]:
# 显示每个集合中的记录数量
for key, value in sorted(feed['Schedule'].items()):
  print('{:3} {}'.format(len(value), key))  # 显示各个集合中的记录数量

  1 conferences
494 events
357 speakers
 53 venues


In [ ]:
# 获取最后一个演讲者的名字
feed['Schedule']['speakers'][-1]['name']

'Carina C. Zona'

In [ ]:
# 获取最后一个演讲者的编号
feed['Schedule']['speakers'][-1]['serial']

141590

In [ ]:
feed['Schedule']['events'][40]['name']

'There *Will* Be Bugs'

In [ ]:
# 每个事件都有一个 'speakers' 字段,列出 0 个或多个演讲者的编号
feed['Schedule']['events'][40]['speakers']

[3471, 5199]

### 19.1.1 使用动态属性访问 JSON 类数据

- 仅当无法使用常规的方式获取属性(即在实例、类或超类中找不到指定的属性), 解释器才会调用特殊的 `__getattr__` 方法

###### 示例 19-5 explore0.py:把一个 JSON 数据集转换成一个嵌套着 `FrozenJSON` 对象、列表和简单类型的 `FrozenJSON` 对象

- 这个类能递归,自动处理嵌套的映射和列表

In [ ]:
%%writefile ch19/explore0.py
from collections import abc


class FrozenJSON:
  """
  一个只读接口，使用属性表示法访问 JSON 类对象
  """
  def __init__(self, mapping):
    self.__data = dict(mapping)  # 确保传入的是字典(或者是能转换成字典的对象, 同时为了安全起见,创建一个副本

  def __getattr__(self, name):  # 仅当无法使用常规的方式获取属性(即在实例、类或超类中找不到指定的属性), 解释器才会调用 __getattr__ 方法
    if hasattr(self.__data, name):  # 如果 name 是实例属性 __data 的属性,返回那个属性。调用字典的 keys 等方法就是通过这种方式处理的
      return getattr(self.__data, name)
    else:
      return FrozenJSON.build(self.__data[name])  # 否则,从 self.__data 中获取 name 键对应的元素,返回调用 FrozenJSON.build() 方法得到的结果

  @classmethod
  def build(cls, obj):  # 一个备选构造方法,@classmethod 装饰器经常这么用
    if isinstance(obj, abc.Mapping):  # obj 是映射,那就构建一个 FrozenJSON 对象
      return cls(obj)
    elif isinstance(obj, abc.MutableSequence):  # 如果是 MutableSequence 对象,必然是列表, 因此,我们把 obj 中的每个元素递归地传给 .build() 方法,构建一个列表
      return [cls.build(item) for item in obj]
    else:  # 如果既不是字典也不是列表,那么原封不动地返回元素
      return obj

Writing ch19/explore0.py


###### 示例 19-4 示例 19-5 定义的 `FrozenJSON` 类能读取属性,如 `name`,还能调用方法,如 `.keys()` 和 `.items()`

In [ ]:
from ch19.osconfeed import load
from ch19.explore0 import FrozenJSON

In [ ]:
raw_feed = load()
feed = FrozenJSON(raw_feed)

In [ ]:
# FrozenJSON 实例能使用属性表示法遍历嵌套的字典;这里,我们获取演讲者列表的元素数量
len(feed.Schedule.speakers) 

357

In [ ]:
# 可以使用底层字典的方法
sorted(feed.Schedule.keys())

['conferences', 'events', 'speakers', 'venues']

In [ ]:
# 显示每个集合中的记录数量
for key, value in sorted(feed.Schedule.items()):
  print('{:3} {}'.format(len(value), key))  # 显示各个集合中的记录数量

  1 conferences
494 events
357 speakers
 53 venues


In [ ]:
# 列表,例如 feed.Schedule.speakers,仍是列表;但是,如果里面的元素是映射,会转换成 FrozenJSON 对象
feed.Schedule.speakers[-1].name

'Carina C. Zona'

In [ ]:
# events 列表中的 40 号元素是一个 JSON 对象,现在则变成一个 FrozenJSON 实例
talk = feed.Schedule.events[40]
type(talk)

ch19.explore0.FrozenJSON

In [ ]:
talk.name

'There *Will* Be Bugs'

In [ ]:
# 事件记录中有一个 speakers 列表,列出演讲者的编号
talk.speakers

[3471, 5199]

In [ ]:
# 读取不存在的属性会抛出 KeyError 异常,而不是通常抛出的 AttributeError 异常
talk.flavor

KeyError: ignored

### 19.1.2 处理无效属性名

In [ ]:
grad = FrozenJSON({'name': 'Jim Bo', 'class': 1982})

In [ ]:
# 此时无法读取 grad.class 的值,因为在 Python 中 class 是保留字
grad.class

SyntaxError: ignored

In [ ]:
# 便仍可通过 getattr 获取
getattr(grad, 'class')

1982

###### 示例 19-6 explore1.py:在名称为 Python 关键字的属性后面加上 _

In [ ]:
%%writefile ch19/explore1.py
from collections import abc
import keyword


class FrozenJSON:
  """
  一个只读接口，使用属性表示法访问 JSON 类对象
  """
  def __init__(self, mapping):
    self.__data = {}
    for key, value in mapping.items():
      if keyword.iskeyword(key):
        key += '_'
      self.__data[key] = value

  def __getattr__(self, name):  # 仅当没有指定名称(name)的属性时才调用 __getattr__ 方法
    if hasattr(self.__data, name):  # 如果 name 是实例属性 __data 的属性,返回那个属性。调用 keys 等方法就是通过这种方式处理的
      return getattr(self.__data, name)
    else:
      return FrozenJSON.build(self.__data[name])  # 否则,从 self.__data 中获取 name 键对应的元素,返回调用 FrozenJSON.build() 方法得到的结果

  @classmethod
  def build(cls, obj):  # 一个备选构造方法,@classmethod 装饰器经常这么用
    if isinstance(obj, abc.Mapping):  # obj 是映射,那就构建一个 FrozenJSON 对象
      return cls(obj)
    elif isinstance(obj, abc.MutableSequence):  # 如果是 MutableSequence 对象,必然是列表, 因此,我们把 obj 中的每个元素递归地传给 .build() 方法,构建一个列表
      return [cls.build(item) for item in obj]
    else:  # 如果既不是字典也不是列表,那么原封不动地返回元素
      return obj

Writing ch19/explore1.py


In [ ]:
from ch19.explore1 import FrozenJSON
grad = FrozenJSON({'name': 'Jim Bo', 'class': 1982})
grad.class_

1982

#### Python 标识符的问题

In [ ]:
# 如果 JSON 对象中的键不是有效的 Python 标识符,也会遇到类似的问题
x = FrozenJSON({'2be': 'or not'})
x.2be

SyntaxError: ignored

- `s.isidentifier()` 方法能根据语言的语法判断 `s` 是否为有效的 Python标识符
  - 对此,有两个解决方法
    - 一个是抛出异常
    - 另一个是把无效的键换成通用名称,例如 attr_0、attr_1,等等

### 19.1.3 使用 `__new__` 方法以灵活的方式创建对象

- 用于构建实例的是特殊方法 `__new__`
  - 这是个类方法(使用特殊方式处理,因此不必使用 @classmethod 装饰器)
  - 必须返回一个实例, 返回的实例会作为第一个参数(即 `self`)传给 `__init__` 方法
  - `__new__` 方法也可以返回其他类的实例
    - 此时,解释器不会调用 `__init__` 方法

- Python 构建对象的过程

```python
# 构建对象的伪代码
def object_maker(the_class, some_arg):
  new_object = the_class.__new__(some_arg)
  if isinstance(new_object, the_class):
    the_class.__init__(new_object, some_arg)
  return new_object


# 下述两个语句的作用基本等效
x = Foo('bar')
x = object_maker(Foo, 'bar')
```

###### 示例 19-7 explore2.py:使用 `__new__` 方法取代 `build` 方法,构建可能是也可能不是 `FrozenJSON` 实例的新对象

In [ ]:
%%writefile ch19/explore2.py
from collections import abc
import keyword


class FrozenJSON:
  """
  一个只读接口，使用属性表示法访问 JSON 类对象
  """

  def __new__(cls, arg):  # __new__ 是类方法,第一个参数是类本身,余下的参数与 __init__ 方法一样,只不过没有 self
    if isinstance(arg, abc.Mapping):
      return super().__new__(cls)  # 默认的行为是委托给超类的 __new__ 方法。这里调用的是 object 基类的 __new__ 方法,把唯一的参数设为 FrozenJSON, 不能传入 arg
    elif isinstance(arg, abc.MutableSequence): 
      return [cls(item) for item in arg]
    else:  # 如果既不是字典也不是列表,那么原封不动地返回元素
      return arg

  def __init__(self, mapping):
    self.__data = {}
    for key, value in mapping.items():
      if keyword.iskeyword(key):
        key += '_'
      self.__data[key] = value

  def __getattr__(self, name):  # 仅当没有指定名称(name)的属性时才调用 __getattr__ 方法
    if hasattr(self.__data, name):  # 如果 name 是实例属性 __data 的属性,返回那个属性。调用 keys 等方法就是通过这种方式处理的
      return getattr(self.__data, name)
    else:
      return FrozenJSON(self.__data[name])


Overwriting ch19/explore2.py


###### 测试示例 19-7

In [ ]:
from ch19.osconfeed import load
from ch19.explore2 import FrozenJSON

In [ ]:
raw_feed = load()
feed = FrozenJSON(raw_feed)

In [ ]:
# FrozenJSON 实例能使用属性表示法遍历嵌套的字典;这里,我们获取演讲者列表的元素数量
len(feed.Schedule.speakers) 

357

In [ ]:
# 可以使用底层字典的方法
sorted(feed.Schedule.keys())

['conferences', 'events', 'speakers', 'venues']

In [ ]:
# 显示每个集合中的记录数量
for key, value in sorted(feed.Schedule.items()):
  print('{:3} {}'.format(len(value), key))  # 显示各个集合中的记录数量

  1 conferences
494 events
357 speakers
 53 venues


In [ ]:
# 列表,例如 feed.Schedule.speakers,仍是列表;但是,如果里面的元素是映射,会转换成 FrozenJSON 对象
feed.Schedule.speakers[-1].name

'Carina C. Zona'

In [ ]:
# events 列表中的 40 号元素是一个 JSON 对象,现在则变成一个 FrozenJSON 实例
talk = feed.Schedule.events[40]
type(talk)

ch19.explore2.FrozenJSON

In [ ]:
talk.name

'There *Will* Be Bugs'

In [ ]:
# 事件记录中有一个 speakers 列表,列出演讲者的编号
talk.speakers

[3471, 5199]

### 19.1.4 使用 shelve 模块调整 OSCON 数据源的结构

- `shelve` 模块
  - `shelve.open` 高阶函数返回一个 `shelve.Shelf` 实例,这是简单的键值对象数据库,背后由 `dbm` 模块支持
    - `shelve.Shelf` 是 `abc.MutableMapping` 的子类,因此提供了处理映射类型的重要方法
    - `shelve.Shelf` 类还提供了几个管理 I/O 的方法,如 `sync` 和 `close`;它也是一个上下文管理器。
    - 只要把新值赋予键,就会保存键和值
    - 键必须是字符串
    - 值必须是 `pickle` 模块能处理的对象


###### 示例 19-9 schedule1.py:访问保存在 `shelve.Shelf` 对象里的 OSCON 日程数据

In [ ]:
%%writefile ch19/schedule1.py
import warnings

import ch19.osconfeed as osconfeed

DB_NAME = 'ch19/data/schedule1_db'
CONFERENCE = 'conference.115'

class Record:
  def __init__(self, **kwargs):
    self.__dict__.update(kwargs) # ➋ 使用关键字参数传入的属性构建实例的常用简便方式

def load_db(db):
  raw_data = osconfeed.load()
  warnings.warn('loading ' + DB_NAME)
  for collection, rec_list in raw_data['Schedule'].items():
    record_type = collection[:-1] # ➎ record_type 的值是去掉尾部 's' 后的集合名(即把 'events' 变成 'event')
    for record in rec_list:
      key = '{}.{}'.format(record_type, record['serial']) # ➏ 使用 record_type 和 'serial' 字段构成 key。
      record['serial'] = key
      db[key] = Record(**record)  # ➑ 构建 Record 实例,存储在数据库中的 key 键名下

Overwriting ch19/schedule1.py


- 更新实例的 `__dict__` 属性, 把值设为一个映射,能快速地在那个实例中创建一堆属性

###### 示例 19-8 测试 schedule1.py 脚本(见示例 19-9)提供的功能

In [ ]:
from ch19.schedule1 import *
import shelve

In [ ]:
db = shelve.open(DB_NAME)  # shelve.open 函数打开现有的数据库文件,或者新建一个

In [ ]:
# 判断数据库是否填充的简便方法是,检查某个已知的键是否存在
if CONFERENCE not in db:
  load_db(db)

/content/drive/My Drive/Colab Notebooks/fluent_python_notes/ch19/schedule1.py:14: UserWarning: loading ch19/data/schedule1_db
  warnings.warn('loading ' + DB_NAME)


In [ ]:
# 获取一条 speaker 记录
speaker = db['speaker.3471']
type(speaker)  # Recdor 类的实例

ch19.schedule1.Record

In [ ]:
speaker.name, speaker.twitter

('Anna Ravenscroft', 'annaraven')

In [ ]:
db.close()

### 19.1.5 使用特性获取链接的记录

- 目的是自动获取 `event` 记录引用的 `venue` 和 `speaker` 记录
  - 这与 Django ORM 访问 `models.ForeignKey` 字段时所做的事类似
    - 得到的不是键,而是链接的模型对象

- ![UML 类图](https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200725194808.png)
  - `Record`
    - `__init__` 方法与 schedule1.py 脚本(见示例 19-9)中的一样;为了辅助测试,增加了 `__eq__` 方法
  - `DbRecord`
    - `Record` 类的子类
    - 添加了 `__db` 类属性,用于设置和获取 `__db` 属性的 `set_db` 和 `get_db` 静态方法
    - 用于从数据库中获取记录的 fetch类方法,
    - 辅助调试和测试的 `__repr__` 实例方法
  - `Event`
    - `DbRecord` 类的子类,添加了用于获取所链接记录的 `venue` 和 `speakers` 特性,以及特殊的 `__repr__` 方法

###### 示例 19-10  schedule2.py 的 doctest

In [3]:
from ch19.schedule2 import *
import shelve

In [4]:
db = shelve.open(DB_NAME)  # shelve.open 函数打开现有的数据库文件,或者新建一个
# 判断数据库是否填充的简便方法是,检查某个已知的键是否存在
if CONFERENCE not in db:
  load_db(db)

/content/drive/My Drive/Colab Notebooks/fluent_python_notes/ch19/schedule2.py:79: UserWarning: loading ch19/data/schedule2_db
  warnings.warn('loading ' + DB_NAME)


In [5]:
DbRecord.set_db(db)  # ➊ DbRecord 类扩展 Record 类,添加对数据库的支持:为了操作数据库,必须为 DbRecord 提供一个数据库的引用，其所设置的是类属性

In [6]:
event = DbRecord.fetch('event.33950')  # ➋ DbRecord.fetch 类方法能获取任何类型的记录
event  # ➌ event 是 Event 类的实例,而 Event 类扩展 DbRecord 类

<Event 'There *Will* Be Bugs'>

In [ ]:
event.venue  # ➍ event.venue 返回一个 DbRecord 实例

<DbRecord serial='venue.1449'>

In [ ]:
event.venue.name  # ➎ 可以直接找出 event.venue 的名称

'Portland 251'

In [ ]:
for spkr in event.speakers:  # ➏ 还可以迭代 event.speakers 列表,获取表示各位演讲者的DbRecord 对象
  print('{0.serial}: {0.name}'.format(spkr))

speaker.3471: Anna Ravenscroft
speaker.5199: Alex Martelli


###### 示例 19-11 schedule2.py:导入模块,定义常量和增强的 `Record` 类


In [ ]:
%%writefile ch19/schedule2.py
import warnings
import inspect  # ➊ 在 load_db 函数中使用

import ch19.osconfeed as osconfeed

DB_NAME = 'ch19/data/schedule2_db'  # ➋ 因为要存储几个不同类的实例,所以我们要创建并使用不同的数据库文件
CONFERENCE = 'conference.115'


class Record:
  def __init__(self, **kwargs):
    self.__dict__.update(kwargs)

  def __eq__(self, other):  # ➌ __eq__ 方法对测试有重大帮助
    if isinstance(other, Record):
      return self.__dict__ == other.__dict__
    else:
      return NotImplemented


Overwriting ch19/schedule2.py


###### 示例 19-12 schedule2.py: `MissingDatabaseError` 类和 `DbRecord` 类

In [ ]:
%%writefile -a ch19/schedule2.py


class MissingDatabaseError(RuntimeError):
  """需要数据库但没有指定数据库时抛出。"""  # ➊ 自定义的异常通常是标志类,没有定义体。写一个文档字符串,说明异常的用途,比只写一个 pass 语句要好。


class DbRecord(Record):  # ➋ 扩展 Record 类
  __db = None  # ➌ __db 类属性存储一个打开的 shelve.Shelf 数据库引用

  @staticmethod  # ➍ set_db 是静态方法,以此强调不管调用多少次,效果始终一样
  def set_db(db):
    DbRecord.__db = db  # ➎ 即使调用 Event.set_db(my_db),__db 属性仍在 DbRecord 类中设置

  @staticmethod  # ➏ get_db 也是静态方法,因为不管怎样调用,返回值始终是 DbRecord.__db 引用的对象
  def get_db():
    return DbRecord.__db
  
  @classmethod  # ➐ fetch 是类方法,因此在子类中易于定制它的行为
  def fetch(cls, ident):
    db = cls.get_db()
    try:
      return db[ident]  # ➑ 从数据库中获取 ident 键对应的记录
    except TypeError:
      if db is None:  # ➒ 抛出自定义的异常,说明必须设置数据库
        msg = "database not set; call '{}.set_db(my_db)'"
        raise MissingDatabaseError(msg.format(cls.__name__))
      else:  # ➓ 重新抛出 TypeError 异常
        raise

  def __repr__(self):
    if hasattr(self, 'serial'):  # ⓫ 如果记录有 serial 属性,在字符串表示形式中使用
      cls_name = self.__class__.__name__
      return '<{} serial={!r}>'.format(cls_name, self.serial)
    else:
      return super().__repr__()  # ⓬ 调用继承的 __repr__ 方法


Appending to ch19/schedule2.py


###### 示例 19-13 schedule2.py： `Event` 类

In [ ]:
%%writefile -a ch19/schedule2.py


class Event(DbRecord):  # ➊ Event 类扩展 DbRecord 类

  @property
  def venue(self):
    key = 'venue.{}'.format(self.venue_serial)
    return self.__class__.fetch(key)  # ➋ 在 venue 特性中使用 venue_serial 属性构建 key,然后传给继承自 DbRecord 类的 fetch 类方法

  @property
  def speakers(self):
    if not hasattr(self, '_speaker_objs'):  # ➌ speakers 特性检查记录是否有 _speaker_objs 属性
      spkr_serials = self.__dict__['speakers']  # ➍ 如果没有,直接从 __dict__ 实例属性中获取 'speakers' 属性的值,防止无限递归,因为这个特性的公开名称也是 speakers
      fetch = self.__class__.fetch  # ➎ 获取 fetch 类方法的引用
      self._speaker_objs = [fetch('speaker.{}'.format(key))
                            for key in spkr_serials]   # ➏ 使用 fetch 获取 speaker 记录列表,然后赋值给 self._speaker_objs
    return self._speaker_objs  # ➐ 返回前面获取的列表

  def __repr__(self):
    if hasattr(self, 'name'):  # ➑ 如果记录有 name 属性,在字符串表示形式中使用
      cls_name = self.__class__.__name__
      return '<{} {!r}>'.format(cls_name, self.name)
    else:
      return super().__repr__()  # ➒ 否则,调用继承的 __repr__ 方法


Appending to ch19/schedule2.py


- 使用 `self.__class__.fetch(key)`, 而不直接使用 `self.fetch(key)` 的原因
  - 如果事件记录有名为 `fetch` 的键，则 `self.fetch` 获取的是 `fetch` 字段的值,而不是 `Event` 继承自 `DbRecord` 的 `fetch` 类方法
- 从数据中创建实例属性的名称时肯定有可能会引入缺陷
  - 因为类属性(例如方法)可能被遮盖,或者由于意外覆盖现有的实例属性而丢失数据

###### 示例 19-14 schedule2.py：`load_db` 函数

- 可以对照示例 19-1 来理解具体的处理逻辑

In [ ]:
%%writefile -a ch19/schedule2.py


def load_db(db):
  raw_data = osconfeed.load()
  warnings.warn('loading ' + DB_NAME)
  for collection, rec_list in raw_data['Schedule'].items():
    record_type = collection[:-1]  # ➊ 目前,与 schedule1.py 脚本(见示例 19-9)中的 load_db 函数一样
    cls_name = record_type.capitalize()  # ➋ 把 record_type 变量的值首字母变成大写(例如,把 'event' 变成 'Event'),获取可能的类名
    cls = globals().get(cls_name, DbRecord)  # ➌ 从模块的全局作用域中获取那个名称对应的对象;如果找不到对象,使用 DbRecord
    if inspect.isclass(cls) and issubclass(cls, DbRecord):  # ➍ 如果获取的对象是类,而且是 DbRecord 的子类
      factory = cls  # ➎ ......把对象赋值给 factory 变量。因此,factory 的值可能是 DbRecord 的任何一个子类,具体的类取决于 record_type 的值。
    else:
      factory = DbRecord  # ➏ 否则,把 DbRecord 赋值给 factory 变量
    for record in rec_list:  # ➐ 这个 for 循环创建 key,然后保存记录,这与之前一样
      key = '{}.{}'.format(record_type, record['serial'])
      record['serial'] = key
      db[key] = factory(**record)  # ➑ 存储在数据库中的对象由 factory 构建,factory 可能是 DbRecord 类,也可能是根据 record_type 的值确定的某个子类

Appending to ch19/schedule2.py


## 19.2 使用特性验证属性

### 19.2.1 `LineItem` 类第1版:表示订单中商品的类

###### 示例 19-15 bulkfood_v1: 最简单的 `LineItem` 类

In [ ]:
%%writefile ch19/bulkfood_v1.py
class LineItem:

  def __init__(self, description, weight, price):
    self.description = description
    self.weight = weight
    self.price = price

  def subtotal(self):
    return self.weight * self.price


Writing ch19/bulkfood_v1.py


###### 示例 19-16 重量为负值时,金额小计为负值

In [ ]:
from ch19.bulkfood_v1 import LineItem

In [ ]:
raisins = LineItem('Golden raisins', 10, 6.95)
raisins.subtotal()

69.5

In [ ]:
raisins.weight = -20 # 无效输入......
raisins.subtotal() # 无效输出......

-139.0

### 19.2.2 LineItem 类第2版:能验证值的特性

In [ ]:
%%writefile ch19/bulkfood_v2.py
class LineItem:
  def __init__(self, description, weight, price):
    self.description = description
    self.weight = weight  # ➊ 使用特性的设值方法了,确保所创建实例的 weight 属性不能为负值
    self.price = price

  def subtotal(self):
    return self.weight * self.price

  @property  # ➋ @property 装饰读值方法
  def weight(self):  # ➌ 实现特性的方法,其名称都与公开属性的名称一样——weight
    return self.__weight  # ➍ 真正的值存储在私有属性 __weight 中

  @weight.setter  # ➎ 被装饰的读值方法有个 .setter 属性,这个属性也是装饰器;这个装饰器把读值方法和设值方法绑定在一起
  def weight(self, value):
    if value > 0:
      self.__weight = value  # ➏ 如果值大于零,设置私有属性 __weight
    else:
      raise ValueError('value must be > 0')  # ➐ 否则,抛出 ValueError 异常


Overwriting ch19/bulkfood_v2.py


In [ ]:
from ch19.bulkfood_v2 import LineItem

In [ ]:
walnuts = LineItem('walnuts', 0, 10.00)

ValueError: ignored

## 19.3 特性全解析

- `property` 是一个类, 其构造方法的完整签名如下：
  - `property(fget=None, fset=None, fdel=None, doc=None)`
    - 所有的方法都是可选的
    - 如果没有把函数传给某个参数,那么得到的特性对象就不允许执行相应的操作

###### 示例 19-18 bulkfood_v2b.py:效果与示例 19-17 一样, 只不过没使用装饰器

In [ ]:
%%writefile ch19/bulkfood_v2b.py
class LineItem:

  def __init__(self, description, weight, price):
    self.description = description
    self.weight = weight
    self.price = price
  
  def subtotal(self):
    return self.weight * self.price
  
  def get_weight(self):  # ➊ 普通的读值方法
    return self.__weight

  def set_weight(self, value):  # ➋ 普通的设值方法
    if value > 0:
      self.__weight = value
    else:
      raise ValueError('value must be > 0')

  weight = property(get_weight, set_weight)  # ➌ 构建 property 对象,然后赋值给公开的类属性

Writing ch19/bulkfood_v2b.py


In [ ]:
from ch19.bulkfood_v2b import LineItem

In [ ]:
walnuts = LineItem('walnuts', 0, 10.00)

ValueError: ignored

### 19.3.1 特性会覆盖实例属性

- 如果实例和所属的类有同名数据属性,那么实例属性会覆盖(或称遮盖)类属
- 但实例属性不会遮盖类特性
  - 但仍然可通过修改 `__dict__` 字典向实例中加入与特性同名的属性

###### 示例 19-19 实例属性遮盖类的数据属性

In [ ]:
class Class: # ➊ 定义 Class 类,这个类有两个类属性:data 数据属性和 prop 特性
  data = 'the class data attr'

  @property
  def prop(self):
    return 'the prop value'

In [ ]:
obj = Class()
vars(obj)  # ➋ vars 函数返回 obj 的 __dict__ 属性,表明没有实例属性

{}

In [ ]:
obj.data  # ➌ 读取 obj.data,获取的是 Class.data 的值

'the class data attr'

In [ ]:
obj.data = 'bar'  # ❹ 为 obj.data 赋值,创建一个实例属性
vars(obj)  # ❺ 审查实例,查看实例属性

{'data': 'bar'}

In [ ]:
obj.data  # ❻ 现在读取 obj.data,获取的是实例属性的值。从 obj 实例中读取属性时,实例属性 data 会遮盖类属性 data。

'bar'

In [ ]:
Class.data  # ❼ Class.data 属性的值完好无损

'the class data attr'

###### 示例 19-20 实例属性不会遮盖类特性(接续示例 19-19)

In [ ]:
Class.prop  # ❶ 直接从 Class 中读取 prop 特性,获取的是特性对象本身,不会运行特性的读值方法

In [ ]:
obj.prop  # ❷ 读取 obj.prop 会执行特性的读值方法

'the prop value'

In [ ]:
obj.prop = 'foo'  # ❸ 尝试设置 prop 实例属性,结果失败

AttributeError: ignored

In [ ]:
obj.__dict__['prop'] = 'foo'  # ➍ 但是可以直接把 'prop' 存入 obj.__dict__
vars(obj)  # ❺ obj 现在有两个实例属性:data 和 prop

{'data': 'bar', 'prop': 'foo'}

In [ ]:
obj.prop  # ❻ 然而,读取 obj.prop 时仍会运行特性的读值方法。特性没被实例属性遮盖

'the prop value'

In [ ]:
Class.prop = 'baz'  # ❼ 覆盖 Class.prop 特性,销毁特性对象

In [ ]:
obj.prop  # ❽ 现在,obj.prop 获取的是实例属性。Class.prop 不是特性了,因此不会再覆盖 obj.prop

'foo'

###### 示例 19-21 新添的类特性遮盖现有的实例属性(接续示例 19-20)


In [ ]:
obj.data  # ❶ obj.data 获取的是实例属性 data

'bar'

In [ ]:
Class.data  # ❷ Class.data 获取的是类属性 data

'the class data attr'

In [ ]:
Class.data = property(lambda self: 'the "data" prop value')  # ❸ 使用新特性覆盖 Class.data
obj.data  # ❹ 现在,obj.data 被 Class.data 特性遮盖了

'the "data" prop value'

In [ ]:
del Class.data  # ❺ 删除特性
obj.data  # ❻ 现在恢复原样,obj.data 获取的是实例属性 data

'bar'

### 19.3.2 特性的文档

- 控制台中的 `help()` 函数或 IDE 等工具需要显示特性的文档时,会从特性的 `__doc__` 属性中提取信息
- 经典调用句法，为 `property` 设置文档字符串的方法是传入 `doc` 参数
  - `weight = property(get_weight, set_weight, doc='weight in kilograms')`
- 使用装饰器创建 `property` 对象时,读值方法(有`@property` 装饰器的方法)的文档字符串作为一个整体,变成特性的文档

###### 示例 19-22 特性的文档

In [ ]:
class Foo:

  @property
  def bar(self):
    '''The bar attribute'''
    return self.__dict__['bar']

  @bar.setter
  def bar(self, value):
    self.__dict__['bar'] = value


In [ ]:
help(Foo)

Help on class Foo in module __main__:

class Foo(builtins.object)
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  bar
 |      The bar attribute



In [ ]:
help(Foo.bar)

Help on property:

    The bar attribute



## 19.4 定义一个特性工厂函数

###### 示例 19-23, 19-24 bulkfood_v2prop.py

In [ ]:
%%writefile ch19/bulkfood_v2prop.py
def quantity(storage_name):  # ➊ storage_name 参数确定各个特性的数据存储在哪儿;对 weight 特性来说,存储的名称是 'weight'
  def qty_getter(instance):  # ➋ qty_getter 函数的第一个参数可以命名为 self, 因为 qty_getter 函数不在类定义体中, 所以将其命名为 instance;instance 指代要把属性存储其中的 LineItem 实例
    return instance.__dict__[storage_name]  # ➌ qty_getter 引用了 storage_name,把它保存在这个函数的闭包里;值直接从 instance.__dict__ 中获取,为的是跳过特性,防止无限递归

  def qty_setter(instance, value):  # ➍ 定义 qty_setter 函数,第一个参数也是 instance
    if value > 0:
      instance.__dict__[storage_name] = value  # ➎ 值直接存到 instance.__dict__ 中, 目的是跳过特性，防止无限递归
    else:
      raise ValueError('value must be > 0')

  return property(qty_getter, qty_setter)


class LineItem:
  weight = quantity('weight')  # ➊ 使用工厂函数把第一个自定义的特性 weight 定义为类属性
  price = quantity('price')  # ➋ 第二次调用,构建另一个自定义的特性,price

  def __init__(self, description, weight, price):
    self.description = description
    self.weight = weight  # ➌ 这里,特性已经激活,确保不能把 weight 设为负数或零
    self.price = price
    
  def subtotal(self):
    return self.weight * self.price  # ➍ 这里也用到了特性,使用特性获取实例中存储的值


Overwriting ch19/bulkfood_v2prop.py


- 每次调用 `quantity` 工厂函数构建属性时,都要把 `storage_name` 参数设为独一无二的值
- 使用 `property` 对象包装 `qty_getter` 和 `qty_setter` 函数
  - 需要运行这两个函数时,它们会从闭包中读取 `storage_name`, 确定从哪里获取属性的值,或者在哪里存储属性的值

###### 示例 19-25 bulkfood_v2prop.py:quantity 特性工厂函数

In [ ]:
from ch19.bulkfood_v2prop import LineItem

In [ ]:
nutmeg = LineItem('Moluccan nutmeg', 8, 13.95)
nutmeg.weight, nutmeg.price  # ➊ 通过特性读取 weight 和 price,这会遮盖同名实例属性

(8, 13.95)

In [ ]:
sorted(vars(nutmeg).items())  # ➋ 使用 vars 函数审查 nutmeg 实例,查看真正用于存储值的实例属性

[('description', 'Moluccan nutmeg'), ('price', 13.95), ('weight', 8)]

In [ ]:
nutmeg.price = -10

ValueError: ignored

## 19.5 处理属性删除操作

- `@my_propety.deleter` 装饰器包装一个方法, 负责删除特性管理的属性
- 在不使用装饰器的经典调用句法中, `fdel` 参数用于设置删值函数

  ```python
  member = property(member_getter, fdel=member_deleter)
  ```

###### 示例 19-26 blackknight.py:灵感来自电影《巨蟒与圣杯》中的黑衣骑士角色

In [ ]:
%%writefile -a ch19/blackknight.py
class BlackKnight:

  def __init__(self):
    self.members = ['an arm', 'another arm',
                    'a leg', 'another leg']
    self.phrases = ["'Tis but a scratch.",
                    "It's just a flesh wound.",
                    "I'm invincible!",
                    "All right, we'll call it a draw."]

  @property
  def member(self):
    print('next member is:')
    return self.members[0]
  
  @member.deleter
  def member(self):
    text = 'BLACK KNIGHT (loses {})\n-- {}'
    print(text.format(self.members.pop(0), self.phrases.pop(0)))


Writing ch19/blackknight.py


###### 示例 19-27 blackknight.py:示例 19-26 的 doctest(黑衣骑士从不屈服)


In [3]:
from ch19.blackknight import BlackKnight
knight = BlackKnight()

In [4]:
knight.member

next member is:


'an arm'

In [5]:
del knight.member

BLACK KNIGHT (loses an arm)
-- 'Tis but a scratch.


In [6]:
del knight.member

BLACK KNIGHT (loses another arm)
-- It's just a flesh wound.


In [7]:
del knight.member

BLACK KNIGHT (loses a leg)
-- I'm invincible!


In [8]:
del knight.member

BLACK KNIGHT (loses another leg)
-- All right, we'll call it a draw.


In [9]:
del knight.member

IndexError: ignored

## 19.6 处理属性的重要属性和函数

### 19.6.1 影响属性处理方式的特殊属性

- 后面的很多函数和特殊方法,其行为受下述 3 个特殊属性的影响

#### `__class__`

- 对象所属类的引用(即 `obj.__class__` , 与 `type(obj)` 的作用相同)
- Python 的某些特殊方法,例如 `__getattr__`, 只在对象的类中寻找,而不在实例中寻找

#### `__dict__`

- 一个映射, 存储对象或类的可写属性。有 `__dict__` 属性的对象, 任何时候都能随意设置新属性
- 如果类有 `__slots__` 属性, 它的实例可能没有 `__dict__` 属性

#### `__slots__`

- 类可以定义这个这属性,限制实例能有哪些属性
- `__slots__` 属性的值是一个字符串组成的元组, 指明允许有的属性
- 如果 `__slots__` 中没有 `__dict__`, 那么该类的实例没有 `__dict__` 属性, 实例只允许有指定名称的属性

### 19.6.2 处理属性的内置函数

- 以下 5 个内置函数对对象的属性做读、写和内省操作

#### `dir([object])`

- 列出对象的大多数属性
  - `dir` 函数的目的是交互式使用,因此没有提供完整的属性列表,只列出一组“重要的”属性名
- `dir` 函数能审查有或没有 `__dict__` 属性的对象
  - `dir` 函数不会列出 `__dict__` 属性本身,但会列出其中的键
- `dir` 函数也不会列出类的几个特殊属性
  - 例如 `__mro__`、`__bases__` 和 `__name__`
- 如果没有指定可选的 `object` 参数,`dir` 函数会列出当前作用域中的名称


#### `getattr(object, name[, default])`

- 从 `object` 对象中获取 `name` 字符串对应的属性
- 获取的属性可能来自对象所属的类或超类
- 如果没有指定的属性
  - `getattr` 函数抛出`AttributeError` 异常
  - 或者返回 `default` 参数的值(如果设定了这个参数的话)


#### `hasattr(object, name)`

- 如果 `object` 对象中存在指定的属性,或者能以某种方式(例如继承)通过 `object` 对象获取指定的属性,返回 `True`
- 这个函数的实现方法是调用 `getattr`(`object`, `name`) 函数,看看是否抛出`AttributeError` 异常


#### `setattr(object, name, value)`

- 把 `object` 对象指定属性的值设为 `value`
- 前提是 `object` 对象能接受那个值。这个函数可能会创建一个新属性,或者覆盖现有的属性

#### `vars([object])`

- 返回 `object` 对象的 `__dict__` 属性
  - 如果实例所属的类定义了`__slots__` 属性,实例没有 `__dict__` 属性,那么 `vars` 函数不能处理那个实例
  - 相反, `dir` 函数能处理这样的实例
- 如果没有指定参数,那么 `vars()` 函数的作用与 `locals()` 函数一样:返回表示本地作用域的字典


### 19.6.3 处理属性的特殊方法

- 使用点号或内置的 `getattr`、`hasattr` 和 `setattr` 函数存取属性都会触发下述列表中相应的特殊方法
- 直接通过实例的 `__dict__` 属性读写属性不会触发这些特殊方法
  - 如果需要,通常会使用这种方式跳过特殊方法
- 特殊方法 `__getattribute__` 和 `__setattr__` 不管怎样都会调用, 几乎会影响每一次属性存取
  - 因此比`__getattr__` 方法(只处理不存在的属性名)更难正确使用
  - 与定义这些特殊方法相比, 使用特性或描述符相对不易出错


- 假设有个名为 `Class` 的类, `obj` 是 `Class` 类的实例, `attr` 是 `obj` 的属性


#### `__delattr__(self, name)`

- 只要使用 `del` 语句删除属性,就会调用这个方法
  - 如, `del obj.attr` 语句触发 `Class.__delattr__(obj, attr)` 方法

#### `__dir__(self)`

- 把对象传给 `dir` 函数时调用, 列出属性
  - `dir(obj)` 触发 `Class.__dir__(obj)` 方法

#### `__getattr__(self, name)`

- 仅当在`obj`、`Class` 和超类中找不到指定的属性时才会触发
  - 表达式 `obj.no_such_attr`、`getattr(obj, 'no_such_attr')` 和 `hasattr(obj, 'no_such_attr')` 在搜索过 `obj`、`Class` 和超类之后，没有找到相关属性，便会触发 `Class.__getattr__(obj, 'no_such_attr')` 方法


#### `__getattribute__(self, name)`

- 尝试获取指定的属性时总会调用这个方法
  - 不过,寻找的属性是特殊属性或特殊方法时除外
- 点号与 `getattr` 和 `hasattr` 内置函数会触发这个方法
- 调用 `__getattribute__` 方法且抛出 `AttributeError` 异常时,才会调用 `__getattr__` 方法
- 为了在获取 `obj` 实例的属性时不导致无限递归, `__getattribute__` 方法的实现要使用`super().__getattribute__(obj, name)`
  - [参考](https://stackoverflow.com/questions/371753/how-do-i-implement-getattribute-without-an-infinite-recursion-error/371833)

#### `__setattr__(self, name, value)`

- 尝试设置指定的属性时总会调用这个方法
- 点号和 `setattr` 内置函数会触发这个方法
  - `obj.attr = 42` 和 `setattr(obj,'attr', 42)` 都会触发 `Class.__setattr__(obj, 'attr', 42)` 方法
